In [1]:
#!pip install seaborn
#!pip install -U scikit-learn
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Preprosesing

In [2]:
from datetime import timedelta

df=pd.read_csv('harth/S006.csv')
df.head()

,timestamp,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label
0,2019-01-12 00:00:00.000,-0.760242,0.299570,0.468570,-5.092732,-0.298644,0.709439,6
1,2019-01-12 00:00:00.010,-0.530138,0.281880,0.319987,0.900547,0.286944,0.340309,6
2,2019-01-12 00:00:00.020,-1.170922,0.186353,-0.167010,-0.035442,-0.078423,-0.515212,6
3,2019-01-12 00:00:00.030,-0.648772,0.016579,-0.054284,-1.554248,-0.950978,-0.221140,6
4,2019-01-12 00:00:00.040,-0.355071,-0.051831,-0.113419,-0.547471,0.140903,-0.653782,6


In [3]:
# Assuming 'df' is your DataFrame and 'columns_to_describe' is a list of column names you want to describe
columns_to_describe = ['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z']

# Select the specific columns and compute descriptive statistics
df[columns_to_describe].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
back_x,408709.0,-0.802201,0.238347,-3.542889,-0.983647,-0.937195,-0.654541,0.952109
back_y,408709.0,-0.000687,0.189062,-3.016498,0.001063,0.033240,0.074822,2.569339
back_z,408709.0,-0.274718,0.441805,-1.024363,-0.702338,-0.277446,0.064811,1.628023
thigh_x,408709.0,-0.370317,0.506666,-6.844045,-0.952840,-0.277711,0.068999,3.898547
thigh_y,408709.0,0.143471,0.213864,-5.757406,0.022534,0.086248,0.246292,4.602909
thigh_z,408709.0,0.617527,0.536430,-4.884791,0.144114,0.924066,1.001372,5.391660


In [4]:
lagged = df.copy()
lagged_columns = []

for i in range(1, 51):
    for col in ['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z']:
        lagged_col_name = f"{col}_{i}"  # Constructing the lagged column name
        lagged[lagged_col_name] = lagged[col].shift(i)  # Creating lagged columns
        lagged_columns.append(lagged_col_name)
# Fill NaN values with 0
lagged.fillna(0, inplace=True)

# Copy the label column from the original DataFrame to lagged_train
lagged['label'] = df['label']

# Display the head of the lagged_train DataFrame to verify the changes
lagged.head()


C:\Users\karag\AppData\Local\Temp\ipykernel_9640\3756131078.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  lagged[lagged_col_name] = lagged[col].shift(i)  # Creating lagged columns
C:\Users\karag\AppData\Local\Temp\ipykernel_9640\3756131078.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  lagged[lagged_col_name] = lagged[col].shift(i)  # Creating lagged columns
C:\Users\karag\AppData\Local\Temp\ipykernel_9640\3756131078.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

,timestamp,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label,back_x_1,back_y_1,...,back_z_49,thigh_x_49,thigh_y_49,thigh_z_49,back_x_50,back_y_50,back_z_50,thigh_x_50,thigh_y_50,thigh_z_50
0,2019-01-12 00:00:00.000,-0.760242,0.299570,0.468570,-5.092732,-0.298644,0.709439,6,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-12 00:00:00.010,-0.530138,0.281880,0.319987,0.900547,0.286944,0.340309,6,-0.760242,0.299570,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-12 00:00:00.020,-1.170922,0.186353,-0.167010,-0.035442,-0.078423,-0.515212,6,-0.530138,0.281880,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-12 00:00:00.030,-0.648772,0.016579,-0.054284,-1.554248,-0.950978,-0.221140,6,-1.170922,0.186353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-12 00:00:00.040,-0.355071,-0.051831,-0.113419,-0.547471,0.140903,-0.653782,6,-0.648772,0.016579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
lagged.head()

,timestamp,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label,back_x_1,back_y_1,...,back_z_49,thigh_x_49,thigh_y_49,thigh_z_49,back_x_50,back_y_50,back_z_50,thigh_x_50,thigh_y_50,thigh_z_50
0,2019-01-12 00:00:00.000,-0.760242,0.299570,0.468570,-5.092732,-0.298644,0.709439,6,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-12 00:00:00.010,-0.530138,0.281880,0.319987,0.900547,0.286944,0.340309,6,-0.760242,0.299570,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-12 00:00:00.020,-1.170922,0.186353,-0.167010,-0.035442,-0.078423,-0.515212,6,-0.530138,0.281880,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-12 00:00:00.030,-0.648772,0.016579,-0.054284,-1.554248,-0.950978,-0.221140,6,-1.170922,0.186353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-12 00:00:00.040,-0.355071,-0.051831,-0.113419,-0.547471,0.140903,-0.653782,6,-0.648772,0.016579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
n = len(lagged)
train = lagged[0:int(n*0.8)]
test = lagged[int(n*0.8):]

# Split lagged_test into features (X_train) and target variable (y_train)
x_train = train.drop(['timestamp', 'label'], axis=1)
y_train = train['label']

# Split lagged_test into features (X_test) and target variable (y_test)
x_test = test.drop(['timestamp', 'label'], axis=1)
y_test = test['label']


In [ ]:
sc= StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# Train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=30, verbose=30)
clf.fit(x_train, y_train)

building tree 1 of 30


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:   42.3s


building tree 2 of 30


[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:  1.6min


building tree 3 of 30


...

building tree 7 of 30


[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:  5.4min


building tree 8 of 30


[Parallel(n_jobs=1)]: Done   8 tasks      | elapsed:  6.3min


building tree 9 of 30


In [12]:
from sklearn.metrics import accuracy_score
# Predict labels for the test set
y_pred = clf.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100)

MemoryError: Unable to allocate 189. MiB for an array with shape (81642, 606) and data type float32

# Random forest Classifier

In [9]:
X_Test = sc.fit_transform(X_Test)
pre =  RFC_model.predict(X_Test)

[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done   2 tasks      | elapsed:    2.4s
[Parallel(n_jobs=1)]: Done   3 tasks      | elapsed:    3.9s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    5.4s
[Parallel(n_jobs=1)]: Done   5 tasks      | elapsed:    6.7s
[Parallel(n_jobs=1)]: Done   6 tasks      | elapsed:    8.1s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:    9.7s
[Parallel(n_jobs=1)]: Done   8 tasks      | elapsed:   11.2s
[Parallel(n_jobs=1)]: Done   9 tasks      | elapsed:   12.5s
[Parallel(n_jobs=1)]: Done  10 tasks      | elapsed:   13.9s
[Parallel(n_jobs=1)]: Done  11 tasks      | elapsed:   15.2s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:   16.8s
[Parallel(n_jobs=1)]: Done  13 tasks      | elapsed:   18.7s
[Parallel(n_jobs=1)]: Done  14 tasks      | elapsed:   20.5s
[Parallel(n_jobs=1)]: Done  15 tasks      | elapsed:   22.2s
[Parallel(n_jobs=1)]: Done  16 tasks      | elapsed:   24.0s
[Parallel(n_jobs=1)]: Do

In [10]:
from sklearn.metrics import accuracy_score

In [11]:
accuracy_score(Y_Test, pre)

0.8519835183571597